In [1]:
import os
from dotenv import load_dotenv

load_dotenv() 
open_ai_key = os.getenv("OPEN_AI_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
youtube_api_key = os.getenv("YOUTUBE_API_KEY")

Python-dotenv could not parse statement starting at line 2


In [2]:
from langchain_openai import ChatOpenAI

def get_llm(api_key:str):
    model = ChatOpenAI(temperature=0, model="gpt-4o-mini", api_key=api_key)
    return model

### Agent Chain 생성
- agent는 
- 관련 정보를 찾을 query문을 다듬어줌
- AI 관련된 내용만을 질문하도록 제한을 걸어줌
- 올바른 매체를 선택하도록 제한해줌 (프롬프팅 엔지니어링 추가)

In [23]:
from pydantic import BaseModel, Field  # 데이터 검증과 직렬화를 위한 Pydantic 라이브러리
from typing import Literal  # 다양한 타입 힌팅 클래스들

class AgentAction(BaseModel):
    """
    에이전트의 행동을 정의하는 Pydantic 모델
    Pydantic은 데이터 검증 및 관리를 위한 라이브러리입니다.
    """
    # Literal을 사용하여 action 필드가 가질 수 있는 값을 제한합니다
    action: Literal["video", "news", "not_supported"] = Field(
        description="에이전트가 수행할 행동의 타입을 지정합니다",
    )
    
    action_input: str = Field(
        description="사용자가 입력한 원본 질의 텍스트입니다",
        min_length=1,  # 최소 1글자 이상이어야 함
    )
    
    search_keyword: str = Field(
        description="""검색에 사용할 최적화된 키워드입니다.
        AI 관련 키워드일 경우 핵심 검색어를 포함하고,
        not_supported 액션의 경우 빈 문자열('')을 사용합니다""",
        examples=["ChatGPT tutorial", "머신러닝 입문 강의"]  # 예시 제공
    )

In [ ]:
from langchain_core.output_parsers import JsonOutputParser  # LLM의 출력을 JSON 형식으로 파싱하는 도구
from langchain_core.prompts import PromptTemplate 

output_parser = JsonOutputParser(pydantic_object=AgentAction)
# print("출력 포맷 가이드 :", output_parser.get_format_instructions())


prompt = PromptTemplate(
            input_variables=["input"],  # 템플릿에서 사용할 변수들
            partial_variables={"format_instructions": output_parser.get_format_instructions()},
            template="""당신은 AI 관련 YouTube 영상을 검색하는 도우미입니다.
입력된 질의가 AI 관련 내용인지 먼저 확인하세요.

AI 관련 주제 판단 기준:
- AI 기술 및 정보 (머신러닝, 딥러닝, 자연어처리 등)
- AI 도구 및 서비스 (ChatGPT, DALL-E, Stable Diffusion 등)
- AI 교육 및 학습
- AI 정책 및 동향

AI 관련 질의가 아닌 경우:
- action을 "not_supported"로 설정
- search_keyword는 빈 문자열로 설정

AI 관련 질의인 경우:
1. action을 "news" 또는 "video" 중에서 선택하세요.

- "video":
  - "영상", "비디오", "동영상"이라는 단어가 포함된 경우
- "news":
    - 분석, 배경 지식이 중요하거나 "뉴스"라는 단어가 포함된 경우.
    
예제:
  - "비전 프로에 관련된 영상을 찾아줘" → "video"
  - "비전 프로의 최근 소식을 알려줘" → "news"
  - "긴급한 비전 프로 발표 영상을 보여줘" → "video"
  - "비전 프로에 대한 분석 기사를 찾아줘" → "news"
  - "gpt 관련 영상을 추천해줘" → "video"

출력은 반드시 아래 형식으로 작성하세요:
  "news" 또는 "video"
     
단어 분석:
  1. "영상", "비디오", "동영상" 또는 이와 유사한 단어가 포함되면 항상 "video"를 선택합니다.
  2. 위 단어가 없으면 "news"를 선택합니다.
    
2. 검색 키워드 최적화:
   - 핵심 주제어 추출
   - 불필요한 단어 제거 (동영상, 찾아줘 등)
   - 전문 용어는 그대로 유지

분석할 질의: {input}

{format_instructions}""")

출력 포맷 가이드 : The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "에이전트의 행동을 정의하는 Pydantic 모델\nPydantic은 데이터 검증 및 관리를 위한 라이브러리입니다.", "properties": {"action": {"description": "에이전트가 수행할 행동의 타입을 지정합니다", "enum": ["video", "news", "not_supported"], "title": "Action", "type": "string"}, "action_input": {"description": "사용자가 입력한 원본 질의 텍스트입니다", "minLength": 1, "title": "Action Input", "type": "string"}, "search_keyword": {"description": "검색에 사용할 최적화된 키워드입니다.\n        AI 관련 키워드일 경우 핵심 검색어를 포함하고,\n        not_supported 액션의 경우 빈 문자열('')을 사용합니다", "examples": ["ChatGPT tutorial", "머신러닝 입문 강의"

In [25]:
def extract_assistant(order:str):
    llm = get_llm(open_ai_key)

    extract_chain = prompt | llm | output_parser
    extract = extract_chain.invoke({"input": order})

    print(f"🧚 {extract.get("action")} 매체로 원하시는 정보를 보여줄게요!")
    print("===============================================")

    return extract

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate

# choose_prompt = ChatPromptTemplate.from_messages([
#     ("system", """
#     아래 지시사항에 따라 "news" 또는 "video"를 선택하세요.
     
#     [지시사항]
#     quiz: {{quiz}}
    
#     - "video":
#       - "영상", "비디오", "동영상"이라는 단어가 포함된 경우
#     - "news":
#       - 분석, 배경 지식이 중요하거나 "뉴스"라는 단어가 포함된 경우.
    
#     예제:
#     - "비전 프로에 관련된 영상을 찾아줘" → "video"
#     - "비전 프로의 최근 소식을 알려줘" → "news"
#     - "긴급한 비전 프로 발표 영상을 보여줘" → "video"
#     - "비전 프로에 대한 분석 기사를 찾아줘" → "news"
#     - "gpt 관련 영상을 추천해줘" → "video"

#     출력은 반드시 아래 형식으로 작성하세요:
#     "news" 또는 "video"
     
#     단어 분석:
#     1. "영상", "비디오", "동영상" 또는 이와 유사한 단어가 포함되면 항상 "video"를 선택합니다.
#     2. 위 단어가 없으면 "news"를 선택합니다.
    
#     """)
# ])


In [ ]:
# def choose_media(order:str):
#     llm = get_llm(open_ai_key)

#     media_chain = choose_prompt | llm
#     media = media_chain.invoke({"quiz": order}).content

#     return media

In [80]:
order = input("질문을 입력해주세요>>>")

print("=============================")
print(extract_assistant(order))

{'action': 'news', 'action_input': 'gpt 관련 개념을 알 수 있는 기사를 알려줘', 'search_keyword': 'gpt 개념'}


### Youtube API agent 생성

- Agent의 역할은 LLM 외부와 통신하며 정보를 획득하거나 상호 작용하는 단계
- 조회 순으로 5개의 비디오를 보여지게 함

In [26]:
from googleapiclient.discovery import build
from datetime import datetime
import pytz

# 유튜브 API 호출 함수
def search_youtube_videos(query: str) -> str:
    api_key = youtube_api_key  # 유튜브 API 키 입력
    youtube = build("youtube", "v3", developerKey=api_key)

    # 유튜브에서 검색
    search_response = youtube.search().list(
        q=query,
        part="snippet",
        maxResults=5,  # 최대 검색 결과 수
        type="video",  # 비디오만 검색
        order="viewCount"  # 조회수 순으로 정렬
    ).execute()

    # 검색 결과 정리
    results = []
    for item in search_response.get("items", []):
        title = item["snippet"]["title"]
        description = item["snippet"]["description"]
        video_id = item["id"]["videoId"]
        video_url = f"https://www.youtube.com/watch?v={video_id}"

        # 비디오 정보 조회
        video_response = youtube.videos().list(
            id=video_id,
            part="snippet,statistics"
        ).execute()

        snippet = video_response["items"][0]["snippet"]
        statistics = video_response["items"][0]["statistics"]
        likes_count = statistics.get("likeCount", 0)
        view_count = statistics.get("viewCount", 0)

        short_description = (description[:150] + '...') if len(description) > 150 else description

        utc_time = snippet["publishedAt"]
        utc_time_dt = datetime.fromisoformat(utc_time.replace("Z", "+00:00"))
        kst_tz = pytz.timezone('Asia/Seoul')
        kst_time = utc_time_dt.astimezone(kst_tz).strftime('%Y-%m-%d %H:%M:%S')


        # 결과 리스트에 추가
        results.append({
            "title": title,
            "url": video_url,
            "description": short_description,
            "likes": likes_count,
            "views": view_count,
            "upload_time": kst_time
        })

    return results

In [33]:
from typing import List


def print_videos_information(videos:List):

    cnt = 0
    for result in videos:
        title = result['title']
        video_url = result['url']
        description = result['description']
        likes = result['likes']
        views = result['views']
        upload_time = result["upload_time"]
        cnt += 1
        
        print(f"{cnt}.")
        print(f"Title: {title}")
        print(f"URL: {video_url}")
        print(f"Description: {description}")
        print(f"Likes: {likes}")
        print(f"Views: {views}")
        print(f"Upload_time: {upload_time}")
        print("===============================================")

In [ ]:
from langchain.vectorstores import Chroma
import chromadb
from langchain_openai import OpenAIEmbeddings

def queryDB(query):
    client = chromadb.HttpClient(host="localhost", port=9000)
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=open_ai_key)
    db = Chroma(client=client, collection_name = "articles1", embedding_function = embeddings)
    # idx를 기준으로 유사도가 있는 문서를 갖고옴
    docs = db.similarity_search_with_score(query)
    return docs

In [29]:
query = "gpt 관련 뉴스"
result = queryDB(query)
result

[(Document(metadata={'sub': '2024.06.14 18:05', 'title': '애플, \'챗GPT\' 탑재 현금 보상 없어..."시리 통합 자체가 보상"', 'url': 'https://www.aitimes.com/news/articleView.html?idxno=160627'}, page_content="애플이 음성비서 '시리(Siri)'에 '챗GPT'를 도입하기로 했지만, 이에 따른 금전적 대가를 오픈AI에 지불하지는 않을 것이란 보도가 나왔다. 블룸버그는 13일(현지시간) 애플과 오픈AI가 기술 파트너십을 체결했으나, 이 계약에 금전적인 대가는 오가지 않았다고 소개했다. 애플은 지난 10일 세계개발자회의(WWDC)에서 시리에 챗GPT를 통합한다고"),
  0.3405027389526367),
 (Document(metadata={'sub': '2024.08.21 07:00', 'title': '역시 고양이는 못 참지!', 'url': 'https://www.aitimes.com/news/articleView.html?idxno=162657'}, page_content='AI타임스 조예주 기자 joyejuoffice@aitimes.com'),
  0.36103561190109423),
 (Document(metadata={'sub': '2024.02.14 18:00', 'title': "챗GPT, 채팅 내용 '기억 설정' 가능해진다", 'url': 'https://www.aitimes.com/news/articleView.html?idxno=157156'}, page_content="오픈AI가 '챗GPT'의 기억력을 강화한다. 사용자와의 과거 대화 내용을 기억, 향후 채팅에 활용해 보다 개인화된 서비스를 제공하겠다는 의도다. 구글이 첨단 멀티모달모델(LMM) ‘제미나이 울트라’를 탑재한 챗봇 ‘제미나이 어드밴스드’를 출시한 데 따라 반격에 나선 것으로 풀이된다. 오픈AI는 13일(현지시간) 블로그를 통해 사용자의 과거 대화 기억 기

In [38]:
def print_news_information(results: List):
    # 정렬: 결과를 (score, Document) 튜플로 변환 후 score에 따라 내림차순으로 정렬
    sorted_results = sorted(results, key=lambda x: x[1], reverse=True)
    
    cnt = 0
    for result in sorted_results:
        document = result[0]
        metadata = document.metadata
        title = metadata.get('title', 'N/A')
        url = metadata.get('url', 'N/A')
        sub = metadata.get('sub', 'N/A')
        page_content = document.page_content
        score = result[1]
        cnt += 1
        
        print(f"{cnt}.")
        print(f"Title: {title}")
        print(f"URL: {url}")
        print(f"Content: {page_content}")
        print(f"Other: {sub}")
        print(f"Relevance: {score}")
        print("===============================================")


In [39]:
print_news_information(result)

1.
Title: 이스트에이드, 콘텐츠 플랫폼 '피키캐스트' '봉봉' 인수...AI 비즈니스 가속화
URL: https://www.aitimes.com/news/articleView.html?idxno=161579
Content: 이스트소프트의 자회사인 이스트에이드(대표 김남현)는 콘텐츠 큐레이션 플랫폼 '피키캐스트'와 인터랙티브 콘텐츠 플랫폼 '봉봉'을 인수하고 개방형 인공지능(AI) 포털로 성장하겠다고 15일 밝혔다. 피키캐스트는 개성 있는 모바일 콘텐츠로 월간 사용자(MAU)가 500만명을 넘어서는 등 인기를 누렸다. 봉봉은 '신이 나를 만들 때' ‘소울메이트’ 등
Other: 박수빈 기자
Relevance: 0.3902859389781952
2.
Title: 챗GPT, 채팅 내용 '기억 설정' 가능해진다
URL: https://www.aitimes.com/news/articleView.html?idxno=157156
Content: 오픈AI가 '챗GPT'의 기억력을 강화한다. 사용자와의 과거 대화 내용을 기억, 향후 채팅에 활용해 보다 개인화된 서비스를 제공하겠다는 의도다. 구글이 첨단 멀티모달모델(LMM) ‘제미나이 울트라’를 탑재한 챗봇 ‘제미나이 어드밴스드’를 출시한 데 따라 반격에 나선 것으로 풀이된다. 오픈AI는 13일(현지시간) 블로그를 통해 사용자의 과거 대화 기억 기능을
Other: 2024.02.14 18:00
Relevance: 0.379759281873703
3.
Title: 역시 고양이는 못 참지!
URL: https://www.aitimes.com/news/articleView.html?idxno=162657
Content: AI타임스 조예주 기자 joyejuoffice@aitimes.com
Other: 2024.08.21 07:00
Relevance: 0.36103561190109423
4.
Title: 애플, '챗GPT' 탑재 현금 보상 없어..."시리 통합 자체가 보상"
URL: https://www.aitimes.com/n

In [42]:
query = input("질문을 입력해주세요 >>>")
print(f"🔍 '{query}' 에 대한 정보를 검색중 .....\n\n")
print("===============================================")

extract = extract_assistant(query)

# action : 사용할 매체
media = extract.get("action")
# search_keyword : 검색할 쿼리
keyword = extract.get("search_keyword")

if media == "video":
    results = search_youtube_videos(keyword)
    print_videos_information(results)

elif media == "news":
    results = queryDB(keyword)
    print_news_information(results)

else:
    print("UNSUPPORTED ACCESS")
    


🔍 'gpt에 관한 문서를 알려줘' 에 대한 정보를 검색중 .....


🧚 news 매체로 원하시는 정보를 보여줄게요!
1.
Title: 파루인쇄전자, GMP 인증 획득..."온열 브랜드에서 헬스케어로 확장"
URL: https://www.aitimes.com/news/articleView.html?idxno=160691
Content: 면상발열 솔루션 전문 파루인쇄전자는 '원적외선 목 어깨 온열찜질기'가 의료기기 제조 및 품질관리 기준 적합 인정서(GMP)를 획득했다고 18일 밝혔다. 파루인쇄전자는 독자적으로 보유한 면상발열 솔루션을 기반으로 절전, 초슬림, 초경량, 친환경을 만족하는 온열제품을 개발·제조·판매하는 히팅 솔루션 전문 기업이다. 자체 온열 브랜드인 '잉코(INKO)'를 통해
Other: 임대준 기자
Relevance: 0.5052315592765808
2.
Title: 챗GPT, 채팅 내용 '기억 설정' 가능해진다
URL: https://www.aitimes.com/news/articleView.html?idxno=157156
Content: 오픈AI가 '챗GPT'의 기억력을 강화한다. 사용자와의 과거 대화 내용을 기억, 향후 채팅에 활용해 보다 개인화된 서비스를 제공하겠다는 의도다. 구글이 첨단 멀티모달모델(LMM) ‘제미나이 울트라’를 탑재한 챗봇 ‘제미나이 어드밴스드’를 출시한 데 따라 반격에 나선 것으로 풀이된다. 오픈AI는 13일(현지시간) 블로그를 통해 사용자의 과거 대화 기억 기능을
Other: 2024.02.14 18:00
Relevance: 0.4806564152240753
3.
Title: "미국 학생·교사, 급속도로 AI 채택 확대...75~80%가 챗GPT 활용"
URL: https://www.aitimes.com/news/articleView.html?idxno=160656
Content: 미국 교육 분야에서 인공지능(AI) 채택이 빠르게 확대되는 것으로 알려졌다. 한 설문 조사에 따르면 미국 교사와 학생 75~80%가